In [1]:
import time
import numpy as np

import mss
import cv2
from PIL import Image

import matplotlib.pyplot as plt

from xpc3 import *
from xpc3_helper import *

In [3]:
client = XPlaneConnect()
reset(client, dtpInit=322.0)

# Useful Functions

In [4]:
def get_pixel_line(A, B, sh, sw):
    # Convert to homogeneous and get Plucker
    Ah = np.append(np.array(A), 1.0)
    Bh = np.append(np.array(B), 1.0)
    L = np.outer(Ah, Bh) - np.outer(Bh, Ah)

    # Get projection matrix
    mv = np.reshape(client.getDREF("sim/graphics/view/world_matrix"), (4, 4)).T
    proj = np.reshape(client.getDREF(
        "sim/graphics/view/projection_matrix_3d"), (4, 4)).T
    proj_3 = np.zeros((3, 4))
    proj_3[:2, :] = proj[:2, :]
    proj_3[2, :] = proj[3, :]
    screen_mat = np.array([[0.5 * sw, 0.0, 0.5 * sw],
                           [0.0, -0.5 * sh, 0.5 * sh],
                           [0.0, 0.0, 1.0]])
    P = screen_mat @ proj_3 @ mv

    lx = P @ L @ P.T
    return np.array([lx[2, 1], lx[0, 2], lx[1, 0]])


def get_plane(A, B, C):
    v1 = A - B
    v2 = C - A
    n = np.cross(v1, v2)
    k = -np.dot(A, n)
    return np.array([n[0], n[1], n[2], k])


def get_plane_int(p1, p2):
    p1_normal = p1[:3]
    p2_normal = p2[:3]

    p3_normal = np.cross(p1_normal, p2_normal)
    det = np.sum(np.square(p3_normal))

    r_point = (np.cross(p3_normal, p2_normal) *
               p1[3] + np.cross(p1_normal, p3_normal) * p2[3]) / det
    r_normal = p3_normal

    return r_point, r_normal


def get_pixels(line, sw):
    a, b, c = line
    xs = np.arange(0, sw, 1)
    ys = [-(a / b) * x + (-c / b) for x in xs]
    return xs, ys


def plot_line_in_image(ss, A, B):
    sh, sw, _ = ss.shape
    l = get_pixel_line(A, B, sh, sw)
    xs, ys = get_pixels(l, sw)
    plt.figure(figsize=(10, 6))
    plt.imshow(ss)
    plt.plot(xs, ys, c='lime', linewidth=2.0)
    plt.axis('off')

# Get Anchor Values

In [5]:
screen_shot = mss.mss()
ss = cv2.cvtColor(np.array(screen_shot.grab(
    screen_shot.monitors[2])), cv2.COLOR_BGRA2BGR)[:, :, ::-1]
sh, sw, _ = ss.shape

In [6]:
H0 = np.reshape(client.getDREF("sim/graphics/view/world_matrix"), (4, 4)).T
proj = np.reshape(client.getDREF(
    "sim/graphics/view/projection_matrix_3d"), (4, 4)).T
proj_3 = np.zeros((3, 4))
proj_3[:2, :] = proj[:2, :]
proj_3[2, :] = proj[3, :]
screen_mat = np.array([[0.5 * sw, 0.0, 0.5 * sw],
                       [0.0, -0.5 * sh, 0.5 * sh],
                       [0.0, 0.0, 1.0]])
K0 = screen_mat @ proj_3

print("H0: ", H0)
print("K0: ", K0)

H0:  [[ 5.92020035e-01 -1.74897327e-03  8.05921376e-01 -1.22593750e+04]
 [-3.43610090e-03  9.99983072e-01  4.69423505e-03 -4.67814697e+02]
 [-8.05915952e-01 -5.54830814e-03  5.92004001e-01 -3.98963867e+04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
K0:  [[ 1.14408348e+03  0.00000000e+00 -9.60000000e+02  0.00000000e+00]
 [ 0.00000000e+00 -1.14408334e+03 -5.40000000e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.00000000e+00  0.00000000e+00]]


In [7]:
l1 = np.array([-23954.87792969, 225.3943862, 32806.69628906])
l2 = np.array([-24449.80410156, 224.0555954, 33161.92558594])
r1 = np.array([-23937.09082031, 225.32855225, 32831.61230469])
r2 = np.array([-24432.01777344, 223.98096924, 33186.75800781])

In [8]:
r1c = H0 @ np.append(r1, 1.0)
r2c = H0 @ np.append(r2, 1.0)
l1c = H0 @ np.append(l1, 1.0)
l2c = H0 @ np.append(l2, 1.0)
c1c = (l1c[:3] + r1c[:3]) / 2

In [9]:
ground_plane = get_plane(np.array(r1c[:3]), np.array(r2c[:3]), np.array(l1c[:3]))

# Rotation

In [80]:
# Things needed
np.set_printoptions(suppress=True)
c1c, u1, ground_plane, K0


(array([   13.38632798,    -6.11540012, -1170.11564816]),
 array([-0.01113786,  0.00331633,  0.99993247]),
 array([    6.92491392, 18648.60891881,   -61.77202323, 41670.59509746]),
 array([[ 1144.08348083,     0.        ,  -960.        ,     0.        ],
        [    0.        , -1144.08333778,  -540.        ,     0.        ],
        [    0.        ,     0.        ,    -1.        ,     0.        ]]))

In [10]:
def get_rot_y(theta):
    return np.array([[np.cos(theta), 0.0, np.sin(theta)],
                     [0.0, 1.0, 0.0],
                     [-np.sin(theta), 0.0, np.cos(theta)]])

In [13]:
def get_corresponding_plane(l, K0):
    return K0.T @ l


def get_3d_line(l, K0, ground_plane):
    edge_plane = get_corresponding_plane(l, K0)
    r_point, r_normal = get_plane_int(edge_plane, ground_plane)
    return r_point, r_normal


def get_distance(p1, l1, p2):
    n = l1 / np.linalg.norm(l1)
    m = np.cross(p1, n)

    d = np.linalg.norm(np.cross(p2, n) - m)

    return d

In [75]:
def get_state(pixel_line, K0, ground_plane, u1, c1):
    p2, u2 = get_3d_line(pixel_line, K0, ground_plane)

    # Determine heading
    cos_heading = np.dot(u1, u2) / (np.linalg.norm(u1) * np.linalg.norm(u2))
    if cos_heading > 1.0:
        cos_heading = 1.0
    heading = np.arccos(cos_heading)  # degrees

    # Unrotate
    Ry = get_rot_y(heading)
    p2rot, u2rot = Ry @ p2, Ry @ u2
    if np.linalg.norm(np.cross(u1, u2rot / np.linalg.norm(u2rot))) > 1e-2:
        # Rotated the wrong way
        heading = -heading
        Ry = get_rot_y(heading)
        p2rot, u2rot = Ry @ p2, Ry @ u2

    # Determine crosstrack
    crosstrack = 13 - get_distance(p2rot, u2rot, c1)

    return crosstrack, np.degrees(heading)


In [82]:
setHomeState(client, 0, 680, 0)
time.sleep(0.2)
line_right = get_pixel_line(r1, r2, sh, sw)
line_right_a, line_right_u = get_3d_line(line_right, K0, ground_plane)
print((r1c[:3] - line_right_a) / line_right_u)
line_left = get_pixel_line(l1, l2, sh, sw)
line_left_a, line_left_u = get_3d_line(line_left, K0, ground_plane)
print((l1c[:3] - line_left_a) / line_left_u)

[-0. -0. -0.]
[-0. -0. -0.]


In [83]:
u1, u2 = line_right_u / \
    np.linalg.norm(line_right_u), line_right_u / np.linalg.norm(line_right_u)

In [85]:
print(get_distance(line_right_a, line_right_u, c1c))
print(get_distance(line_left_a, line_left_u, c1c))

12.875368850141637
12.958430400724165


In [79]:
setHomeState(client, 0.0, 680, 0.0)
time.sleep(0.2)
line_right = get_pixel_line(r1, r2, sh, sw)
line_left = get_pixel_line(l1, l2, sh, sw)
get_state(line_left, K0, ground_plane, u1, c1c)

(0.008017141968711172, 0.0016428193500113806)

In [47]:
get_state(line_left, K0, ground_plane, u1, c1c)

(6.022070910518487, 0.0024267446215457547)

In [84]:
# Things needed
np.set_printoptions(suppress=True)
c1c, u1, ground_plane, K0

(array([   13.38632798,    -6.11540012, -1170.11564816]),
 array([-0.01113786,  0.00331633,  0.99993247]),
 array([    6.92491392, 18648.60891881,   -61.77202323, 41670.59509746]),
 array([[ 1144.08348083,     0.        ,  -960.        ,     0.        ],
        [    0.        , -1144.08333778,  -540.        ,     0.        ],
        [    0.        ,     0.        ,    -1.        ,     0.        ]]))